In [1]:
import time
import pandas as pd
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTQzNTIwMzYsImlhdCI6MTc2NTk1MjAzNiwibmJmIjoxNzY1OTUyMDM2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCI3NzljMTAyNy03ZDQ1LTRlOWItYWM5ZS1iNDgzMWRiODQzZTFcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjllODBhNjM2LTY4OGMtNDQ4OC1hMDhjLTU1NzQwMDQwNDMwZlwiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OmM5ODU6OWEzNjo2ZTEyOjFjZWIsMTYyLjE1OC4yMzUuMjA0LDM1LjI0MS4yMy4xMjNcIixcInR3b0ZhRXhwaXJ5VHNcIjoyNTU0MzUyMDM2MzA0fSIsImlzcyI6ImFwZXgtYXV0aC1wcm9kLWFwcCJ9.vFYYnOrSLi-teVY6qhFF11SeSVZRIo-xBz_lVlOoTDujYw3ucWZSbOoP9sqFg11Oc8cCwWASqbg_R-9BfmPU0Q"
API_SECRET = "Gb!4#@-d4*XbNz)F!)Y0iW8122uJiaTn"

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"Auth failed: {e}")
        exit()

groww = auth()
print("Logged into Groww!")

Ready to Groww!
Logged into Groww!


In [4]:
# --- CONFIGURATION ---
# The new method requires these specific strings
SYMBOL = "NSE-NIFTY" 
EXCHANGE = "NSE"          # Passed as string
SEGMENT = "CASH"       # Passed as string ("INDICES" for Nifty 50, "CASH" for stocks)
INTERVAL = "5minute"           # Passed as string (e.g., "1m", "5m", "15m")

# --- CHUNKING LOGIC ---
total_days = 2
chunk_size_days = 5
end_date = pd.Timestamp.now()
start_date = end_date - timedelta(days=total_days)

all_candles = []
current_start = start_date

print(f"Starting batched download for {SYMBOL} ({SEGMENT})...")

while current_start < end_date:
    current_end = current_start + timedelta(days=chunk_size_days)
    if current_end > end_date:
        current_end = end_date
    
    # Format dates to string as required by signature
    s_str = current_start.strftime("%Y-%m-%d %H:%M:%S")
    e_str = current_end.strftime("%Y-%m-%d %H:%M:%S")
    
    print(f"Fetching: {s_str} -> {e_str}")
    
    try:
        # UPDATED CALL MATCHING YOUR SIGNATURE
        resp = groww.get_historical_candles(
            exchange=EXCHANGE,
            segment=SEGMENT,
            groww_symbol=SYMBOL,
            start_time=s_str,
            end_time=e_str,
            candle_interval=INTERVAL
        )
        
        # Parse response
        if resp and "candles" in resp and resp["candles"]:
            chunk_data = resp["candles"]
            all_candles.extend(chunk_data)
            print(f"  > Success: {len(chunk_data)} candles.")
        else:
            # Sometimes API returns empty or wrapped data, print to debug if needed
            print(f"  > No data/Empty response. (Check if Market was open)")
            
    except Exception as e:
        print(f"  > Error: {e}")

    current_start = current_end
    time.sleep(0.5) # Rate limit protection

# --- SAVE TO CSV ---
# --- SAVE TO CSV ---
if all_candles:
    # 1. Create DataFrame
    df = pd.DataFrame(all_candles)
    
    # 2. Rename Columns (Handle optional 7th column)
    base_cols = ["timestamp", "open", "high", "low", "close", "volume"]
    rename_map = {i: col for i, col in enumerate(base_cols)}
    df.rename(columns=rename_map, inplace=True)
    
    if len(df.columns) > 6:
        df.rename(columns={6: "open_interest"}, inplace=True)
    
    # 3. ROBUST TIMESTAMP FIX
    # Check the first value to decide how to convert
    first_ts = df["timestamp"].iloc[0]
    
    if isinstance(first_ts, str):
        # Case A: API returns strings like '2025-11-07T13:43:00'
        # Do NOT use unit='s' here.
        df["timestamp"] = pd.to_datetime(df["timestamp"])
    else:
        # Case B: API returns numbers (Epoch seconds) like 1735660000
        # MUST use unit='s' here.
        df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
    
    # 4. Clean & Save
    df.sort_values("timestamp", inplace=True)
    df.drop_duplicates(subset="timestamp", inplace=True)
    
    filename = "nifty_live.csv"
    df.to_csv(filename, index=False)
    print(f"\nDONE! Saved {len(df)} candles to {filename}")
    print("First 5 rows:")
    print(df.head())
else:
    print("\nFAILED: No data collected.")

Starting batched download for NSE-NIFTY (CASH)...
Fetching: 2025-12-15 12:21:02 -> 2025-12-17 12:21:02
  > Success: 166 candles.

DONE! Saved 166 candles to nifty_live.csv
First 5 rows:
            timestamp      open      high       low     close volume  \
0 2025-12-15 12:20:00  26023.05  26025.75  26020.75  26023.40   None   
1 2025-12-15 12:25:00  26023.70  26025.85  26011.45  26018.60   None   
2 2025-12-15 12:30:00  26020.10  26027.65  26017.45  26025.60   None   
3 2025-12-15 12:35:00  26025.85  26029.65  26019.95  26022.20   None   
4 2025-12-15 12:40:00  26021.85  26024.30  26007.70  26010.15   None   

  open_interest  
0          None  
1          None  
2          None  
3          None  
4          None  


In [3]:
# The script already does this:
df.drop(columns=['volume', 'open_interest'], errors='ignore', inplace=True)

In [2]:
import pandas as pd
import ta  # Technical Analysis library (Works on Python 3.14)
from tsfracdiff import FractionalDifferentiator 

# 1. Load your raw data
print("Loading data...")
try:
    df = pd.read_csv('nifty_spot_1m.csv')
except FileNotFoundError:
    print("Error: nifty_spot_1m.csv not found.")
    exit()

# 2. Cleanup
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)
df.drop(columns=['volume', 'open_interest'], errors='ignore', inplace=True)

# 3. Feature Engineering: Technical Indicators (Using 'ta' lib)
print("Generating Technical Indicators...")

# RSI (Momentum)
# Initialize RSI Indicator
rsi_indicator = ta.momentum.RSIIndicator(close=df['close'], window=14)
df['RSI'] = rsi_indicator.rsi()

# MACD (Trend)
# Initialize MACD Indicator
macd_indicator = ta.trend.MACD(close=df['close'])
df['MACD'] = macd_indicator.macd()
df['MACD_Signal'] = macd_indicator.macd_signal()

# Bollinger Bands (Volatility)
# Initialize Bollinger Bands Indicator
bb_indicator = ta.volatility.BollingerBands(close=df['close'], window=20, window_dev=2)
df['BB_High'] = bb_indicator.bollinger_hband()
df['BB_Low'] = bb_indicator.bollinger_lband()

# 4. The "Secret Sauce": Fractional Differentiation
print("Applying Fractional Differentiation (This might take a moment)...")

# We only differentiate the 'close' price for the AI to learn
frac_diff = FractionalDifferentiator()
res = frac_diff.FitTransform(df['close']) 
df['FracDiff_Close'] = res

# 5. Create Targets (What we want to predict)
# Goal: Will price go UP in the next 5 minutes?
df['Target_Return'] = df['close'].shift(-5) - df['close'] 
df['Target'] = (df['Target_Return'] > 0).astype(int)

# Drop NaNs created by indicators/shifting
df.dropna(inplace=True)

# 6. Save Final Training Data
output_file = 'nifty_training_data.csv'
df.to_csv(output_file)
print(f"\nSUCCESS! Processed data saved to {output_file}")
print(f"Features ready: {list(df.columns)}")
print(df.head())

Loading data...
Generating Technical Indicators...
Applying Fractional Differentiation (This might take a moment)...


d:\StockMarket\StockMarket\.venv\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)



SUCCESS! Processed data saved to nifty_training_data.csv
Features ready: ['open', 'high', 'low', 'close', 'RSI', 'MACD', 'MACD_Signal', 'BB_High', 'BB_Low', 'FracDiff_Close', 'Target_Return', 'Target']
                         open      high       low     close        RSI  \
timestamp                                                                
2025-11-07 14:56:00  25500.65  25503.60  25498.95  25500.50  42.884080   
2025-11-07 14:57:00  25499.70  25504.05  25493.05  25493.60  35.420975   
2025-11-07 14:58:00  25493.20  25493.45  25486.75  25488.25  30.926820   
2025-11-07 14:59:00  25487.50  25490.55  25484.60  25490.10  34.043195   
2025-11-07 15:00:00  25492.50  25496.75  25489.75  25492.85  38.486026   

                         MACD  MACD_Signal       BB_High        BB_Low  \
timestamp                                                                
2025-11-07 14:56:00 -2.746876    -1.087647  25521.563339  25492.671661   
2025-11-07 14:57:00 -3.248718    -1.519861  25520.580397

In [ ]:
import pandas as pd
import time
import os
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "YOUR_API_KEY_HERE"
API_SECRET = "YOUR_API_SECRET_HERE"

# --- CONFIGURATION ---
# 1. EXPIRY DATE STRING
# User confirmed format is "09Dec25" (Day + ShortMonth + ShortYear)
# Change this to match your target expiry (e.g., "26Dec25" for monthly)
EXPIRY_DATE_STR = "09Dec25" 

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"Auth failed: {e}")
        exit()

# 1. Load Spot Data
print("Loading Spot Data...")
try:
    spot_df = pd.read_csv('nifty_spot_1m.csv')
    # Standardize timestamp
    if isinstance(spot_df['timestamp'].iloc[0], str):
        spot_df['timestamp'] = pd.to_datetime(spot_df['timestamp'])
    else:
        spot_df['timestamp'] = pd.to_datetime(spot_df['timestamp'], unit='s')
except FileNotFoundError:
    print("Error: nifty_spot_1m.csv not found.")
    exit()

# 2. Identify Unique ATM Strikes
# Round to nearest 50
spot_df['ATM_Strike'] = (spot_df['close'] / 50).round() * 50
unique_strikes = sorted(spot_df['ATM_Strike'].unique().astype(int))

print(f"Found {len(unique_strikes)} unique strikes: {unique_strikes}")
print("-" * 50)

groww = auth()
start_dt = spot_df['timestamp'].min()
end_dt = spot_df['timestamp'].max()

# Create output folder
os.makedirs("options_data", exist_ok=True)

# 3. Download Loop
for strike in unique_strikes:
    # --- NEW SYMBOL GENERATION LOGIC ---
    # Format: NSE-NIFTY-09Dec25-25450-CE
    ce_symbol = f"NSE-NIFTY-{EXPIRY_DATE_STR}-{strike}-CE"
    pe_symbol = f"NSE-NIFTY-{EXPIRY_DATE_STR}-{strike}-PE"
    
    symbols_to_fetch = [ce_symbol, pe_symbol]
    
    for sym in symbols_to_fetch:
        filename = f"options_data/{sym}.csv"
        
        if os.path.exists(filename):
            print(f"Skipping {sym} (Already exists)")
            continue
            
        print(f"Downloading History for {sym}...")
        
        current_start = start_dt
        all_candles = []
        
        while current_start < end_dt:
            current_end = current_start + timedelta(days=5)
            if current_end > end_dt: current_end = end_dt
            
            try:
                # DYNAMIC CALL (No longer hardcoded!)
                resp = groww.get_historical_candles(
                    exchange="NSE",
                    segment="FNO", 
                    groww_symbol=sym,  # <--- Using the dynamic variable here
                    start_time=current_start.strftime("%Y-%m-%d %H:%M:%S"),
                    end_time=current_end.strftime("%Y-%m-%d %H:%M:%S"),
                    candle_interval="1minute"
                )
                
                if resp and "candles" in resp and resp["candles"]:
                    all_candles.extend(resp["candles"])
            
            except Exception as e:
                print(f"  > Error fetching chunk: {e}")
            
            current_start = current_end
            time.sleep(0.5) 
            
        # Save if we got data
        if all_candles:
            df = pd.DataFrame(all_candles)
            
            # Fix Columns
            base_cols = ["timestamp", "open", "high", "low", "close", "volume"]
            rename_map = {i: col for i, col in enumerate(base_cols)}
            df.rename(columns=rename_map, inplace=True)
            if len(df.columns) > 6: df.rename(columns={6: "oi"}, inplace=True)
            
            # Fix Timestamp
            if isinstance(df["timestamp"].iloc[0], str):
                df["timestamp"] = pd.to_datetime(df["timestamp"])
            else:
                df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')

            df.drop_duplicates(subset="timestamp", inplace=True)
            df.to_csv(filename, index=False)
            print(f"  > Saved {len(df)} rows to {filename}")
        else:
            print(f"  > NO DATA for {sym}")

print("\nDownload Complete. Check the 'options_data' folder.")

Loading Spot Data...
Found 19 unique strikes touched: [np.int64(25450), np.int64(25500), np.int64(25550), np.int64(25600), np.int64(25650), np.int64(25700), np.int64(25750), np.int64(25800), np.int64(25850), np.int64(25900), np.int64(25950), np.int64(26000), np.int64(26050), np.int64(26100), np.int64(26150), np.int64(26200), np.int64(26250), np.int64(26300), np.int64(26350)]
--------------------------------------------------
Ready to Groww!
  > Saved 454 rows to options_data/NIFTY25D0925450CE.csv
  > Saved 454 rows to options_data/NIFTY25D0925450PE.csv
  > Saved 454 rows to options_data/NIFTY25D0925500CE.csv


KeyboardInterrupt: 

In [ ]:
import time
import pandas as pd
import datetime
import os
from growwapi import GrowwAPI

API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTM0ODUwMTcsImlhdCI6MTc2NTA4NTAxNywibmJmIjoxNzY1MDg1MDE3LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCJmYjg0YzJmOS04NGUwLTQ2NGMtYWFkZC0wZjMyZTBiNDZmY2FcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjVlZDUwZmU2LTBiNjktNDBlMC04ZDJmLTJlZjE3Y2YxZDYwN1wiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OjFjNDg6YTliYjo5MTZiOmI4NWQsMTcyLjcxLjE5OC4xMjgsMzUuMjQxLjIzLjEyM1wiLFwidHdvRmFFeHBpcnlUc1wiOjI1NTM0ODUwMTc4ODR9IiwiaXNzIjoiYXBleC1hdXRoLXByb2QtYXBwIn0.GCoXAEdA0BkhB88lQmsYqzl96qaGudoM3UvzHxEh_tGfODPmrLzTNPMo8KCeTpzwf46Hp-wU41QxjNPwGyHmag"
API_SECRET = "F@ldixy2hTCYKBq30fyNIyz#PaJ1Ui9i"

# --- CONFIGURATION ---
INDEX_SYMBOL = "NIFTY"
CSV_FILE = "option_ltp_1m.csv"
SLEEP_SECONDS = 60

# Automatically find next Thursday for the symbol format
def get_next_expiry_format():
    """
    Returns the next Thursday in 'DDMonYY' format (e.g., '09Dec25').
    """
    today = datetime.date.today()
    # 0=Mon, 3=Thu
    days_ahead = 3 - today.weekday()
    if days_ahead <= 0: 
        # If today is Thu (0) or later...
        # If today is Thu, use today. If Fri/Sat/Sun, go to next week.
        if days_ahead < 0:
            days_ahead += 7
            
    next_thursday = today + datetime.timedelta(days=days_ahead)
    # Format: 09Dec25 (%d%b%y)
    return next_thursday.strftime("%d%b%y")

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"Auth failed: {e}")
        exit()

def get_nifty_spot(groww_client):
    """Fetches the live Nifty 50 Index value."""
    try:
        # Fetching Index LTP usually requires a specific call or searching 'NSE-NIFTY'
        # Method A: get_ltp with generic symbol
        groww_client.get_ltp()
        resp = groww_client.get_ltp(
            exchange=groww_client.EXCHANGE_NSE,
            segment=groww_client.SEGMENT_INDICES,
            exchange_trading_symbols=[INDEX_SYMBOL]
        )
        # Parse: Resp is usually a list of dicts or a dict
        if isinstance(resp, list) and len(resp) > 0:
            return resp[0].get('ltp')
        elif isinstance(resp, dict):
            # Sometimes returns {'NIFTY': {'ltp': ...}}
            val = list(resp.values())[0]
            if isinstance(val, dict): return val.get('ltp')
            return val
    except Exception as e:
        print(f"Error fetching Spot: {e}")
    return None

def get_option_ltp(groww_client, symbol):
    """Fetches LTP for a specific Option Symbol."""
    try:
        # Using get_ltp for FNO segment
        resp = groww_client.get_ltp(
            exchange=groww_client.EXCHANGE_NSE,
            segment=groww_client.SEGMENT_FNO,
            exchange_trading_symbols=[symbol]
        )
        # Parse logic same as above
        if isinstance(resp, list) and len(resp) > 0:
            return resp[0].get('ltp')
        elif isinstance(resp, dict):
            val = list(resp.values())[0]
            if isinstance(val, dict): return val.get('ltp')
            return val
    except Exception as e:
        # Common error: Symbol doesn't exist (wrong expiry/strike)
        print(f"Error fetching Option {symbol}: {e}")
    return None

# --- MAIN ---
groww = auth()
print("Logged into Groww!")

# Calculate Expiry String once (e.g. "12Dec25")
# Verify this prints the date you expect!
expiry_str = get_next_expiry_format() 
print(f"Targeting Expiry: {expiry_str}")

# Initialize CSV if missing
if not os.path.exists(CSV_FILE):
    pd.DataFrame(columns=["timestamp", "spot_price", "atm_strike", "ce_symbol", "ce_ltp", "pe_symbol", "pe_ltp"]).to_csv(CSV_FILE, index=False)

print(f"Starting Collection... (Saving to {CSV_FILE})")

while True:
    try:
        now_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # 1. Fetch Spot
        spot_ltp = get_nifty_spot(groww)
        
        if spot_ltp:
            # 2. Determine ATM
            atm_strike = round(spot_ltp / 50) * 50
            
            # 3. Construct Symbols
            # Format: NSE-NIFTY-DDMonYY-Strike-Type
            ce_sym = f"NSE-NIFTY-{expiry_str}-{atm_strike}-CE"
            pe_sym = f"NSE-NIFTY-{expiry_str}-{atm_strike}-PE"
            
            # 4. Fetch Option LTPs
            ce_ltp = get_option_ltp(groww, ce_sym)
            pe_ltp = get_option_ltp(groww, pe_sym)
            
            # 5. Save Data
            if ce_ltp is not None and pe_ltp is not None:
                print(f"[{now_str}] Spot: {spot_ltp} | ATM: {atm_strike} | CE: {ce_ltp} | PE: {pe_ltp}")
                
                new_row = {
                    "timestamp": now_str,
                    "spot_price": spot_ltp,
                    "atm_strike": atm_strike,
                    "ce_symbol": ce_sym,
                    "ce_ltp": ce_ltp,
                    "pe_symbol": pe_sym,
                    "pe_ltp": pe_ltp
                }
                
                # Append to CSV
                pd.DataFrame([new_row]).to_csv(CSV_FILE, mode='a', header=False, index=False)
            else:
                print(f"[{now_str}] Error: Could not fetch Option LTPs (Check Expiry/Symbol format)")
        else:
            print(f"[{now_str}] Error: Could not fetch Nifty Spot")

    except Exception as e:
        print(f"Loop Error: {e}")
        
    time.sleep(SLEEP_SECONDS)

Ready to Groww!
Logged into Groww!
Targeting Expiry: 18Dec25
Starting Collection... (Saving to option_ltp_1m.csv)
Error fetching Spot: GrowwAPI.get_ltp() missing 2 required positional arguments: 'exchange_trading_symbols' and 'segment'
[2025-12-17 12:39:15] Error: Could not fetch Nifty Spot


In [ ]:
import pandas as pd
import time
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTQzNTIwMzYsImlhdCI6MTc2NTk1MjAzNiwibmJmIjoxNzY1OTUyMDM2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCI3NzljMTAyNy03ZDQ1LTRlOWItYWM5ZS1iNDgzMWRiODQzZTFcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjllODBhNjM2LTY4OGMtNDQ4OC1hMDhjLTU1NzQwMDQwNDMwZlwiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OmM5ODU6OWEzNjo2ZTEyOjFjZWIsMTYyLjE1OC4yMzUuMjA0LDM1LjI0MS4yMy4xMjNcIixcInR3b0ZhRXhwaXJ5VHNcIjoyNTU0MzUyMDM2MzA0fSIsImlzcyI6ImFwZXgtYXV0aC1wcm9kLWFwcCJ9.vFYYnOrSLi-teVY6qhFF11SeSVZRIo-xBz_lVlOoTDujYw3ucWZSbOoP9sqFg11Oc8cCwWASqbg_R-9BfmPU0Q"
API_SECRET = "Gb!4#@-d4*XbNz)F!)Y0iW8122uJiaTn"

# --- CONFIGURATION ---
SPOT_FILE = "nifty_spot_1m.csv"
OUTPUT_FILE = "option_ltp_1m_fsdfddfdfdfssdfrolling.csv"

# --- USER DEFINED EXPIRY SCHEDULE ---
# We list your specific dates here (Year-Month-Day)
# The script will pick the first date that is >= the current row's date.
EXPIRY_SCHEDULE = [
    pd.Timestamp("2025-12-23")
]

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"Auth failed: {e}")
        exit()

def get_target_expiry(current_ts):
    """
    Finds the correct expiry from the hardcoded list.
    Logic: The nearest date in the list that is >= current_ts.
    """
    current_date = current_ts.date()
    for exp_ts in EXPIRY_SCHEDULE:
        exp_date = exp_ts.date()
        if exp_date >= current_date:
            # Returns format: 04Nov25
            return exp_ts.strftime("%d%b%y")
    return None

# --- STEP 1: PREPARE SPOT DATA ---
print(f"Loading {SPOT_FILE}...")
try:
    df_spot = pd.read_csv(SPOT_FILE)
    if isinstance(df_spot['timestamp'].iloc[0], str):
        df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'])
    else:
        df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'], unit='s')
except FileNotFoundError:
    print(f"Error: {SPOT_FILE} not found.")
    exit()

# 1. Calculate ATM Strike
df_spot['atm_strike'] = (df_spot['close'] / 50).round() * 50

# 2. Assign Correct Expiry to Every Minute
print("Assigning Expiry Dates...")
df_spot['expiry_str'] = df_spot['timestamp'].apply(get_target_expiry)

# Remove rows where no expiry was found (e.g. data after Dec 9)
original_len = len(df_spot)
df_spot.dropna(subset=['expiry_str'], inplace=True)
if len(df_spot) < original_len:
    print(f"Warning: Dropped {original_len - len(df_spot)} rows that were past the last expiry date.")

print("Expiries required:", df_spot['expiry_str'].unique())

# --- STEP 2: DOWNLOAD BATCH HISTORY ---
groww = auth()
option_cache = {} 
grouped = df_spot.groupby('expiry_str')

print("\n--- Downloading Contracts ---")

for expiry, group in grouped:
    unique_strikes = group['atm_strike'].unique().astype(int)
    print(f"\nProcessing Expiry: {expiry} (Unique Strikes: {len(unique_strikes)})")
    
    # We only need data for the time range where this expiry was active
    start_dt = group['timestamp'].min()
    end_dt = group['timestamp'].max()
    
    for strike in unique_strikes:
        ce_symbol = f"NSE-NIFTY-{expiry}-{strike}-CE"
        pe_symbol = f"NSE-NIFTY-{expiry}-{strike}-PE"
        
        for symbol in [ce_symbol, pe_symbol]:
            cache_key = symbol 
            if cache_key in option_cache: continue
            
            print(f"  Fetching {symbol}...", end="")
            
            current_start = start_dt
            all_candles = []
            while current_start < end_dt:
                current_end = current_start + timedelta(days=5)
                if current_end > end_dt: current_end = end_dt
                
                try:
                    resp = groww.get_historical_candles(
                        exchange="NSE",
                        segment="FNO",
                        groww_symbol=symbol,
                        start_time=current_start.strftime("%Y-%m-%d %H:%M:%S"),
                        end_time=current_end.strftime("%Y-%m-%d %H:%M:%S"),
                        candle_interval="1minute"
                    )
                    if resp and "candles" in resp:
                        all_candles.extend(resp["candles"])
                except:
                    pass
                current_start = current_end
                time.sleep(0.1)

            if all_candles:
                temp_df = pd.DataFrame(all_candles)
                if len(temp_df.columns) >= 5:
                    # Rename columns 0->ts, 4->close
                    rename_map = {0: "timestamp", 4: "close"}
                    temp_df.rename(columns=rename_map, inplace=True)
                
                # Timestamp cleanup
                first_ts = temp_df["timestamp"].iloc[0]
                if isinstance(first_ts, str):
                    temp_df["timestamp"] = pd.to_datetime(temp_df["timestamp"])
                else:
                    temp_df["timestamp"] = pd.to_datetime(temp_df["timestamp"], unit='s')
                
                temp_df.set_index("timestamp", inplace=True)
                # Remove duplicates just in case
                temp_df = temp_df[~temp_df.index.duplicated(keep='first')]
                
                option_cache[cache_key] = temp_df
                print(" Done")
            else:
                print(" Failed (No Data)")

# --- STEP 3: STITCH FINAL FILE ---
print(f"\n--- Stitching {OUTPUT_FILE} ---")
final_rows = []

for index, row in df_spot.iterrows():
    ts = row['timestamp']
    nifty_ltp = row['close']
    atm_strike = int(row['atm_strike'])
    expiry = row['expiry_str']
    
    # Construct exact keys
    ce_key = f"NSE-NIFTY-{expiry}-{atm_strike}-CE"
    pe_key = f"NSE-NIFTY-{expiry}-{atm_strike}-PE"
    
    ce_ltp = None
    pe_ltp = None
    
    # Lookup CE
    if ce_key in option_cache:
        # We need to handle potential missing minutes (ffill logic is usually better, but keeping it simple)
        try:
            ce_ltp = option_cache[ce_key].loc[ts]['close']
        except KeyError:
            pass # Data missing for this exact minute
            
    # Lookup PE
    if pe_key in option_cache:
        try:
            pe_ltp = option_cache[pe_key].loc[ts]['close']
        except KeyError:
            pass
            
    final_rows.append({
        "timestamp": ts,
        "nifty_ltp": nifty_ltp,
        "expiry": expiry,
        "atm_strike": atm_strike,
        "atm_ce_ltp": ce_ltp,
        "atm_pe_ltp": pe_ltp
    })

# Save
final_df = pd.DataFrame(final_rows)
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"\nSUCCESS! Saved {len(final_df)} rows to {OUTPUT_FILE}")
print(final_df.head())

Loading nifty_spot_1m.csv...
Assigning Expiry Dates...
Expiries required: ['23Dec25']
Ready to Groww!

--- Downloading Contracts ---

Processing Expiry: 23Dec25 (Unique Strikes: 375)
  Fetching NSE-NIFTY-23Dec25-12200-CE...

In [ ]:
import pandas as pd
import time
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
# Replace with your actual credentials
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTM0ODUwMTcsImlhdCI6MTc2NTA4NTAxNywibmJmIjoxNzY1MDg1MDE3LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCJmYjg0YzJmOS04NGUwLTQ2NGMtYWFkZC0wZjMyZTBiNDZmY2FcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjVlZDUwZmU2LTBiNjktNDBlMC04ZDJmLTJlZjE3Y2YxZDYwN1wiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OjFjNDg6YTliYjo5MTZiOmI4NWQsMTcyLjcxLjE5OC4xMjgsMzUuMjQxLjIzLjEyM1wiLFwidHdvRmFFeHBpcnlUc1wiOjI1NTM0ODUwMTc4ODR9IiwiaXNzIjoiYXBleC1hdXRoLXByb2QtYXBwIn0.GCoXAEdA0BkhB88lQmsYqzl96qaGudoM3UvzHxEh_tGfODPmrLzTNPMo8KCeTpzwf46Hp-wU41QxjNPwGyHmag"
API_SECRET = "F@ldixy2hTCYKBq30fyNIyz#PaJ1Ui9i"

# --- CONFIGURATION ---
SPOT_FILE = "Final_nifty_spot_1minute_fixed_trimmed.csv"
OUTPUT_FILE = "option_ltp.csv"

# --- USER DEFINED EXPIRY SCHEDULE (Tuesdays) ---
# Confirmed: Nifty expiry moves to Tuesday in late 2025.
EXPIRY_SCHEDULE = [
    pd.Timestamp("2025-11-04"),
    pd.Timestamp("2025-11-11"),
    pd.Timestamp("2025-11-18"),
    pd.Timestamp("2025-11-25"), # Monthly Expiry
    pd.Timestamp("2025-12-02"),
    pd.Timestamp("2025-12-09")
]

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"Auth failed: {e}")
        exit()

def get_target_expiry_date(current_ts):
    """Returns the Timestamp of the next expiry."""
    current_date = current_ts.date()
    for exp_ts in EXPIRY_SCHEDULE:
        if exp_ts.date() >= current_date:
            return exp_ts
    return None

# --- STEP 1: PREPARE SPOT DATA ---
print(f"Loading {SPOT_FILE}...")
try:
    df_spot = pd.read_csv(SPOT_FILE)
    if isinstance(df_spot['timestamp'].iloc[0], str):
        df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'])
    else:
        df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'], unit='s')
except FileNotFoundError:
    print(f"Error: {SPOT_FILE} not found.")
    exit()

# 1. Calculate ATM Strike
df_spot['atm_strike'] = (df_spot['close'] / 50).round() * 50

# 2. Assign Expiry Date
df_spot['expiry_dt'] = df_spot['timestamp'].apply(get_target_expiry_date)
df_spot.dropna(subset=['expiry_dt'], inplace=True)

# Generate formatted string for grouping (e.g. 25Nov25)
df_spot['expiry_str'] = df_spot['expiry_dt'].dt.strftime("%d%b%y")

print("Expiries required:", df_spot['expiry_str'].unique())

# --- STEP 2: DOWNLOAD BATCH HISTORY ---
groww = auth()
option_cache = {} 
grouped = df_spot.groupby('expiry_str')

print("\n--- Downloading Contracts ---")

for expiry_str, group in grouped:
    unique_strikes = group['atm_strike'].unique().astype(int)
    print(f"\nProcessing Expiry: {expiry_str} (Strikes: {len(unique_strikes)})")
    
    start_dt = group['timestamp'].min()
    end_dt = group['timestamp'].max()
    
    # Identify if this might be monthly (e.g. Nov 25) to prepare alternate format
    # Alternate format: MONYY (e.g. Nov25)
    # We get this from the timestamp object we stored
    sample_ts = group['expiry_dt'].iloc[0]
    alt_expiry_str = sample_ts.strftime("%b%y") # "Nov25"
    
    for strike in unique_strikes:
        # We will try two formats: Weekly (25Nov25) and Monthly (Nov25)
        # 1. Standard Weekly Format
        ce_weekly = f"NSE-NIFTY-{expiry_str}-{strike}-CE"
        pe_weekly = f"NSE-NIFTY-{expiry_str}-{strike}-PE"
        
        # 2. Monthly Format (Fallback)
        ce_monthly = f"NSE-NIFTY-{alt_expiry_str}-{strike}-CE"
        pe_monthly = f"NSE-NIFTY-{alt_expiry_str}-{strike}-PE"
        
        # Pairs to try: (Standard, Fallback)
        pairs = [(ce_weekly, ce_monthly), (pe_weekly, pe_monthly)]
        
        for std_sym, alt_sym in pairs:
            # Check if we already have either
            if std_sym in option_cache or alt_sym in option_cache:
                continue
            
            # TRY STANDARD
            target_sym = std_sym
            print(f"  Fetching {target_sym}...", end="")
            
            candles = []
            success = False
            
            # Function to fetch data
            def fetch_data(sym):
                c = []
                curr = start_dt
                while curr < end_dt:
                    nxt = curr + timedelta(days=5)
                    if nxt > end_dt: nxt = end_dt
                    try:
                        resp = groww.get_historical_candles(
                            exchange="NSE", segment="FNO", groww_symbol=sym,
                            start_time=curr.strftime("%Y-%m-%d %H:%M:%S"),
                            end_time=nxt.strftime("%Y-%m-%d %H:%M:%S"),
                            candle_interval="1minute"
                        )
                        if resp and "candles" in resp: c.extend(resp["candles"])
                    except Exception as e:
                        pass # Ignore chunk errors
                    curr = nxt
                    time.sleep(0.1)
                return c

            # Attempt 1: Standard
            candles = fetch_data(std_sym)
            
            if not candles:
                # Attempt 2: Fallback (Monthly Format)
                print(f" Failed. Retrying as {alt_sym}...", end="")
                candles = fetch_data(alt_sym)
                if candles:
                    target_sym = alt_sym # Switch to the working symbol
            
            if candles:
                # Process DataFrame
                temp_df = pd.DataFrame(candles)
                if len(temp_df.columns) >= 5:
                    rename_map = {0: "timestamp", 4: "close"}
                    temp_df.rename(columns=rename_map, inplace=True)
                
                if isinstance(temp_df["timestamp"].iloc[0], str):
                    temp_df["timestamp"] = pd.to_datetime(temp_df["timestamp"])
                else:
                    temp_df["timestamp"] = pd.to_datetime(temp_df["timestamp"], unit='s')
                
                temp_df.set_index("timestamp", inplace=True)
                temp_df = temp_df[~temp_df.index.duplicated(keep='first')]
                
                option_cache[target_sym] = temp_df
                print(f" Done ({len(temp_df)} rows)")
            else:
                print(" Failed (No Data)")

# --- STEP 3: STITCH FINAL FILE ---
print(f"\n--- Stitching {OUTPUT_FILE} ---")
final_rows = []

for index, row in df_spot.iterrows():
    ts = row['timestamp']
    nifty_ltp = row['close']
    atm_strike = int(row['atm_strike'])
    expiry_str = row['expiry_str']
    
    # Reconstruct keys to look up
    # Must match logic above (Try Standard, then Fallback)
    
    # Helper to find which symbol exists in cache
    def get_data(std, alt):
        if std in option_cache: return std, option_cache[std]
        if alt in option_cache: return alt, option_cache[alt]
        return None, None

    sample_ts = row['expiry_dt']
    alt_expiry_str = sample_ts.strftime("%b%y") # "Nov25"
    
    ce_std = f"NSE-NIFTY-{expiry_str}-{atm_strike}-CE"
    ce_alt = f"NSE-NIFTY-{alt_expiry_str}-{atm_strike}-CE"
    
    pe_std = f"NSE-NIFTY-{expiry_str}-{atm_strike}-PE"
    pe_alt = f"NSE-NIFTY-{alt_expiry_str}-{atm_strike}-PE"
    
    # CE Lookup
    ce_used, ce_df = get_data(ce_std, ce_alt)
    ce_ltp = ce_df.loc[ts]['close'] if (ce_df is not None and ts in ce_df.index) else None
    
    # PE Lookup
    pe_used, pe_df = get_data(pe_std, pe_alt)
    pe_ltp = pe_df.loc[ts]['close'] if (pe_df is not None and ts in pe_df.index) else None
    
    final_rows.append({
        "timestamp": ts,
        "nifty_ltp": nifty_ltp,
        "expiry": expiry_str,
        "atm_strike": atm_strike,
        "ce_symbol": ce_used if ce_used else ce_std, # Log the symbol used
        "pe_symbol": pe_used if pe_used else pe_std,
        "atm_ce_ltp": ce_ltp,
        "atm_pe_ltp": pe_ltp
    })  

# Save
final_df = pd.DataFrame(final_rows)
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"\nSUCCESS! Saved {len(final_df)} rows to {OUTPUT_FILE}")
print(final_df.head())

Loading nifty_spot_1m.csv...
Expiries required: ['11Nov25' '18Nov25' '25Nov25' '02Dec25' '09Dec25']
Ready to Groww!

--- Downloading Contracts ---

Processing Expiry: 02Dec25 (Strikes: 14)
  Fetching NSE-NIFTY-02Dec25-25600-CE... Done (1408 rows)
  Fetching NSE-NIFTY-02Dec25-25600-PE... Done (1879 rows)
  Fetching NSE-NIFTY-02Dec25-25700-CE... Done (1809 rows)
  Fetching NSE-NIFTY-02Dec25-25700-PE... Done (1879 rows)
  Fetching NSE-NIFTY-02Dec25-25750-CE... Done (1578 rows)
  Fetching NSE-NIFTY-02Dec25-25750-PE... Done (1878 rows)
  Fetching NSE-NIFTY-02Dec25-25800-CE... Done (1873 rows)
  Fetching NSE-NIFTY-02Dec25-25800-PE... Done (1879 rows)
  Fetching NSE-NIFTY-02Dec25-25850-CE... Done (1816 rows)
  Fetching NSE-NIFTY-02Dec25-25850-PE... Done (1878 rows)
  Fetching NSE-NIFTY-02Dec25-25950-CE... Done (1873 rows)
  Fetching NSE-NIFTY-02Dec25-25950-PE... Done (1879 rows)
  Fetching NSE-NIFTY-02Dec25-26000-CE... Done (1879 rows)
  Fetching NSE-NIFTY-02Dec25-26000-PE... Done (1880 rows)

In [13]:
# Load and compare two different CSV files by the same column

# 1. Load both CSV files
df1 = pd.read_csv('option_ltp_1m_rolling.csv')  # Replace with your first CSV file
df2 = pd.read_csv('Final_option_ltp_1m_rolling.csv')  # Replace with your second CSV file

# 2. Compare a specific column (e.g., 'atm_ce_ltp')
column_to_compare = 'atm_pe_ltp'  # Change this to your column name

# Convert to datetime if comparing timestamps
if column_to_compare == 'atm_pe_ltp':
	df1['atm_pe_ltp'] = pd.to_datetime(df1['atm_pe_ltp'])
	df2['atm_pe_ltp'] = pd.to_datetime(df2['atm_pe_ltp'])
	# Calculate time difference in seconds
	df1[f'{column_to_compare}_diff'] = (df1['atm_pe_ltp'] - df2['atm_pe_ltp']).dt.total_seconds()
else:
	# Calculate differences for numeric columns
	df1[f'{column_to_compare}_diff'] = df1[column_to_compare] - df2[column_to_compare]

# 3. Summary statistics
print(f"=== Comparison of '{column_to_compare}' ===")
print(f"Mean Difference: {df1[f'{column_to_compare}_diff'].mean():.2f}")
print(f"Max Difference: {df1[f'{column_to_compare}_diff'].max():.2f}")
print(f"Min Difference: {df1[f'{column_to_compare}_diff'].min():.2f}")
print(f"Std Dev: {df1[f'{column_to_compare}_diff'].std():.2f}")

# 4. View rows with largest differences
print(f"\nTop 10 Largest Differences:")
print(df1.nlargest(10, f'{column_to_compare}_diff')[['atm_pe_ltp', column_to_compare, f'{column_to_compare}_diff']])

# 5. Save comparison
df1[['timestamp', column_to_compare, f'{column_to_compare}_diff']].to_csv('comparison_result.csv', index=False)
print("\nComparison saved to 'comparison_result.csv'")

=== Comparison of 'atm_pe_ltp' ===
Mean Difference: 0.00
Max Difference: 0.00
Min Difference: 0.00
Std Dev: 0.00

Top 10 Largest Differences:
                     atm_pe_ltp                    atm_pe_ltp  atm_pe_ltp_diff
0 1970-01-01 00:00:00.000000072 1970-01-01 00:00:00.000000072              0.0
1 1970-01-01 00:00:00.000000071 1970-01-01 00:00:00.000000071              0.0
2 1970-01-01 00:00:00.000000073 1970-01-01 00:00:00.000000073              0.0
3 1970-01-01 00:00:00.000000093 1970-01-01 00:00:00.000000093              0.0
4 1970-01-01 00:00:00.000000071 1970-01-01 00:00:00.000000071              0.0
5 1970-01-01 00:00:00.000000090 1970-01-01 00:00:00.000000090              0.0
6 1970-01-01 00:00:00.000000088 1970-01-01 00:00:00.000000088              0.0
7 1970-01-01 00:00:00.000000086 1970-01-01 00:00:00.000000086              0.0
8 1970-01-01 00:00:00.000000085 1970-01-01 00:00:00.000000085              0.0
9 1970-01-01 00:00:00.000000086 1970-01-01 00:00:00.000000086       

In [15]:
import pandas as pd
import numpy as np

# --- CONFIGURATION ---
INPUT_FILE = "Final_option_ltp_1m_rolling.csv"  # Or "Final_option_ltp_1m_rolling.csv"
OUTPUT_FILE = "option_ltp_features_1m.csv"

def prepare_features():
    print(f"Reading {INPUT_FILE}...")
    try:
        df = pd.read_csv(INPUT_FILE)
    except FileNotFoundError:
        # Try alternative name
        df = pd.read_csv("Final_option_ltp_1m_rolling.csv")

    # 1. Basic Cleanup
    # Ensure timestamp is datetime
    if not pd.api.types.is_datetime64_any_dtype(df['timestamp']):
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Sort by time to ensure shifts work correctly
    df = df.sort_values('timestamp').reset_index(drop=True)

    # Robustness: Reconstruct Symbols if missing
    # (Format: NSE-NIFTY-{expiry}-{strike}-CE)
    if 'ce_sym' not in df.columns:
        df['ce_sym'] = 'NSE-NIFTY-' + df['expiry'].astype(str) + '-' + df['atm_strike'].astype(str) + '-CE'
    if 'pe_sym' not in df.columns:
        df['pe_sym'] = 'NSE-NIFTY-' + df['expiry'].astype(str) + '-' + df['atm_strike'].astype(str) + '-PE'

    # --- PART 1: FUTURE RETURNS (TARGETS) ---
    print("Computing Future Targets...")
    for k in [1, 3, 5]:
        # Shift BACKWARDS to see into the future (t+k)
        nifty_future = df['nifty_ltp'].shift(-k)
        ce_future = df['atm_ce_ltp'].shift(-k)
        pe_future = df['atm_pe_ltp'].shift(-k)
        
        # Verify time continuity (Ensure we don't jump days)
        time_future = df['timestamp'].shift(-k)
        time_diff = (time_future - df['timestamp']).dt.total_seconds() / 60
        valid_mask = (np.abs(time_diff - k) < 0.1) # Tolerance for float math
        
        # Calculate Returns: (Future / Current) - 1
        df[f'nifty_ret_{k}m'] = np.where(valid_mask, (nifty_future / df['nifty_ltp']) - 1, np.nan)
        df[f'ce_ret_{k}m'] = np.where(valid_mask, (ce_future / df['atm_ce_ltp']) - 1, np.nan)
        df[f'pe_ret_{k}m'] = np.where(valid_mask, (pe_future / df['atm_pe_ltp']) - 1, np.nan)

    # Binary Direction Labels (1 if > 0, else 0)
    df['ce_up_1m'] = (df['ce_ret_1m'] > 0).astype(int)
    df['pe_up_1m'] = (df['pe_ret_1m'] > 0).astype(int)

    # --- PART 2: LAGS & MOMENTUM (PAST FEATURES) ---
    print("Computing Lags & Momentum...")
    
    # Check time continuity for past lag (t-1)
    time_prev = df['timestamp'].shift(1)
    time_diff_prev = (df['timestamp'] - time_prev).dt.total_seconds() / 60
    valid_lag = (np.abs(time_diff_prev - 1) < 0.1)

    # Lag 1 Returns: (Current / Previous) - 1
    # Note: We calculate this manually to handle the valid_lag mask properly
    nifty_ret_past = (df['nifty_ltp'] / df['nifty_ltp'].shift(1)) - 1
    nifty_ret_past[~valid_lag] = np.nan # Mask invalid lags (e.g. first minute of day)

    ce_ret_past = (df['atm_ce_ltp'] / df['atm_ce_ltp'].shift(1)) - 1
    ce_ret_past[~valid_lag] = np.nan

    pe_ret_past = (df['atm_pe_ltp'] / df['atm_pe_ltp'].shift(1)) - 1
    pe_ret_past[~valid_lag] = np.nan

    # Assign Lags
    df['nifty_ret_1m_lag1'] = nifty_ret_past
    df['ce_ret_1m_lag1'] = ce_ret_past
    df['pe_ret_1m_lag1'] = pe_ret_past

    # Rolling Features (Using the past returns calculated above)
    # Rolling Std of 1-min returns (Window=3)
    df['nifty_ret_3m_rolling_std'] = nifty_ret_past.rolling(window=3).std()
    
    # Rolling Mean of Price (Window=5)
    df['nifty_ma_5m'] = df['nifty_ltp'].rolling(window=5).mean()
    
    # Rolling Std of Option Returns (Window=5)
    df['ce_rolling_std_5m'] = ce_ret_past.rolling(window=5).std()
    df['pe_rolling_std_5m'] = pe_ret_past.rolling(window=5).std()

    # Spread Features
    df['ce_pe_mid'] = (df['atm_ce_ltp'] + df['atm_pe_ltp']) / 2
    df['ce_minus_pe'] = df['atm_ce_ltp'] - df['atm_pe_ltp']

    # --- PART 3: TIME FEATURES ---
    print("Computing Time Features...")
    
    # Helper: Minutes from 09:15
    def get_minutes_from_open(dt):
        return (dt.hour * 60 + dt.minute) - (9 * 60 + 15)

    df['minute_of_day'] = df['timestamp'].apply(get_minutes_from_open)
    
    # Opening Hour: 09:15 to 09:45 (0 to 30 mins)
    df['is_opening_hour'] = ((df['minute_of_day'] >= 0) & (df['minute_of_day'] <= 30)).astype(int)
    
    # Closing Hour: After 15:00
    df['is_closing_hour'] = (df['timestamp'].dt.time >= pd.to_datetime('15:00:00').time()).astype(int)

    # --- PART 4: FINAL CLEANUP & SAVE ---
    print("Finalizing...")

    # Define Target Columns (Futures)
    target_cols = [
        'nifty_ret_1m', 'nifty_ret_3m', 'nifty_ret_5m',
        'ce_ret_1m', 'ce_ret_3m', 'ce_ret_5m',
        'pe_ret_1m', 'pe_ret_3m', 'pe_ret_5m'
    ]

    # 1. Drop rows where we don't know the future (Targets are NaN)
    df_final = df.dropna(subset=target_cols).copy()
    
    # 2. Drop rows where we don't know the past (Features are NaN, e.g. first 5 mins of day)
    # This ensures your AI doesn't crash on NaNs
    df_final = df_final.dropna()

    # Define Column Order
    base_cols = ['timestamp', 'expiry', 'atm_strike', 'ce_sym', 'pe_sym', 'nifty_ltp', 'atm_ce_ltp', 'atm_pe_ltp']
    
    feature_cols = [
        'nifty_ret_1m_lag1', 'nifty_ret_3m_rolling_std', 'nifty_ma_5m',
        'ce_ret_1m_lag1', 'pe_ret_1m_lag1',
        'ce_rolling_std_5m', 'pe_rolling_std_5m',
        'ce_pe_mid', 'ce_minus_pe',
        'minute_of_day', 'is_opening_hour', 'is_closing_hour'
    ]
    
    binary_targets = ['ce_up_1m', 'pe_up_1m']
    
    # Combine all
    final_order = base_cols + feature_cols + target_cols + binary_targets
    df_final = df_final[final_order]

    # Save
    df_final.to_csv(OUTPUT_FILE, index=False)
    
    # Print Stats
    print("-" * 30)
    print(f"✅ SUCCESS: Saved to {OUTPUT_FILE}")
    print(f"Total Rows: {len(df_final)}")
    print(f"Proportion CE UP (1m): {df_final['ce_up_1m'].mean():.2%}")
    print(f"Proportion PE UP (1m): {df_final['pe_up_1m'].mean():.2%}")
    print("-" * 30)

if __name__ == "__main__":
    prepare_features()

Reading Final_option_ltp_1m_rolling.csv...
Computing Future Targets...
Computing Lags & Momentum...
Computing Time Features...
Finalizing...
------------------------------
✅ SUCCESS: Saved to option_ltp_features_1m.csv
Total Rows: 7387
Proportion CE UP (1m): 48.57%
Proportion PE UP (1m): 49.05%
------------------------------


In [17]:
# build_features_from_final.py
"""
Reads:  Final_option_ltp_1m_rolling.csv
Writes: /mnt/data/option_ltp_features_1m.csv (or ./option_ltp_features_1m.csv if /mnt/data not present)

Expected columns anywhere in the file (any order):
  timestamp, nifty_ltp, atm_ce_ltp, atm_pe_ltp, expiry, atm_strike, ce_symbol, pe_symbol

Produces forward returns (1/3/5m), binary labels, lag features, rolling stats, time features.
"""
import pandas as pd
from dateutil import tz
from pathlib import Path
import sys

IN_FILE = Path("Final_option_ltp_1m_rolling.csv")      # input (update path if needed)
OUT_FILE = Path("F_option_ltp_features_1m.csv")

if not IN_FILE.exists():
    print(f"Input file not found: {IN_FILE.resolve()}")
    sys.exit(1)

# Read CSV (don't force dtype for all columns to preserve order flexibility)
df = pd.read_csv(IN_FILE)

# Check presence of required columns (names you confirmed)
required = {'timestamp','nifty_ltp','atm_ce_ltp','atm_pe_ltp','expiry','atm_strike','ce_symbol','pe_symbol'}
missing = required - set(df.columns.str.strip())
if missing:
    print("Missing required columns in input CSV:", missing)
    print("Input columns:", list(df.columns))
    sys.exit(1)

# Standardize column names (strip spaces)
df.columns = [c.strip() for c in df.columns]

# Parse timestamp and localize/convert to Asia/Kolkata
KZ = tz.gettz("Asia/Kolkata")
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
if df['timestamp'].isna().any():
    print("Warning: some timestamps could not be parsed. Rows with NaT will be dropped.")
df = df.dropna(subset=['timestamp']).copy()

# If naive datetimes, localize to Asia/Kolkata (assume IST)
if df['timestamp'].dt.tz is None:
    df['timestamp'] = df['timestamp'].dt.tz_localize(KZ)
else:
    df['timestamp'] = df['timestamp'].dt.tz_convert(KZ)

# Ensure numeric price columns
for col in ['nifty_ltp','atm_ce_ltp','atm_pe_ltp','atm_strike']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Sort by timestamp
df = df.sort_values('timestamp').reset_index(drop=True)

# Add market_date (IST date) for grouping by trading day
df['market_date'] = df['timestamp'].dt.tz_convert(KZ).dt.date

# Function to compute features for each market day
def compute_group_features(g):
    g = g.sort_values('timestamp').copy()
    # forward returns: 1,3,5 minutes
    for m in (1,3,5):
        g[f'nifty_ret_{m}m'] = (g['nifty_ltp'].shift(-m) / g['nifty_ltp']) - 1
        g[f'ce_ret_{m}m']    = (g['atm_ce_ltp'].shift(-m) / g['atm_ce_ltp']) - 1
        g[f'pe_ret_{m}m']    = (g['atm_pe_ltp'].shift(-m) / g['atm_pe_ltp']) - 1

    # binary 1-minute up targets
    g['ce_up_1m'] = (g['ce_ret_1m'] > 0).astype(int)
    g['pe_up_1m'] = (g['pe_ret_1m'] > 0).astype(int)

    # lag returns
    g['nifty_ret_1m_lag1'] = g['nifty_ltp'].pct_change(periods=1)
    g['ce_ret_1m_lag1'] = g['atm_ce_ltp'].pct_change(periods=1)
    g['pe_ret_1m_lag1'] = g['atm_pe_ltp'].pct_change(periods=1)

    # rolling stats and moving averages
    g['nifty_ret_3m_rolling_std'] = g['nifty_ltp'].pct_change().rolling(window=3, min_periods=1).std()
    g['nifty_ma_5m'] = g['nifty_ltp'].rolling(window=5, min_periods=1).mean()

    g['ce_rolling_std_5m'] = g['atm_ce_ltp'].pct_change().rolling(window=5, min_periods=1).std()
    g['pe_rolling_std_5m'] = g['atm_pe_ltp'].pct_change().rolling(window=5, min_periods=1).std()

    # straddle mid, skew
    g['ce_pe_mid'] = (g['atm_ce_ltp'] + g['atm_pe_ltp']) / 2.0
    g['ce_minus_pe'] = g['atm_ce_ltp'] - g['atm_pe_ltp']

    return g

# Apply per market day
df_features = df.groupby('market_date', group_keys=False).apply(compute_group_features)

# Time-based features (minute-of-day relative to market open 09:15 IST)
df_features['ts_local'] = df_features['timestamp'].dt.tz_convert(KZ)
market_open = pd.to_datetime(df_features['ts_local'].dt.date.astype(str) + ' 09:15:00').dt.tz_localize(KZ)
df_features['minute_of_day'] = ((df_features['ts_local'] - market_open).dt.total_seconds() / 60).astype('Int64')
df_features['is_opening_hour'] = ((df_features['ts_local'].dt.time >= pd.to_datetime('09:15:00').time()) & (df_features['ts_local'].dt.time < pd.to_datetime('09:45:00').time())).astype(int)
df_features['is_closing_hour'] = (df_features['ts_local'].dt.time >= pd.to_datetime('15:00:00').time()).astype(int)

# Select final columns order: keep basic identity columns first (in your preferred order)
ident_cols = ['timestamp','expiry','atm_strike','ce_symbol','pe_symbol','nifty_ltp','atm_ce_ltp','atm_pe_ltp']
# Ensure identity columns exist (they should)
ident_cols = [c for c in ident_cols if c in df_features.columns]

# Feature / target columns: everything else except helper cols
drop_cols = {'market_date','ts_local'}
all_cols = [c for c in df_features.columns if c not in drop_cols]
# Put ident_cols first, rest afterwards
final_cols = ident_cols + [c for c in all_cols if c not in ident_cols]

final = df_features[final_cols].copy()

# Drop rows missing forward 1-min returns (end-of-day)
final = final.dropna(subset=['ce_ret_1m','pe_ret_1m','nifty_ret_1m']).reset_index(drop=True)

# Save to CSV
final.to_csv(OUT_FILE, index=False)
print("Saved:", OUT_FILE.resolve())
print("Rows:", len(final))

# Print CE/PE up proportions (if present)
if 'ce_up_1m' in final.columns:
    ce_pos = int(final['ce_up_1m'].sum()); ce_total = len(final)
    print(f"CE up 1m: {ce_pos}/{ce_total} = {ce_pos/ce_total:.3f}")
if 'pe_up_1m' in final.columns:
    pe_pos = int(final['pe_up_1m'].sum()); pe_total = len(final)
    print(f"PE up 1m: {pe_pos}/{pe_total} = {pe_pos/pe_total:.3f}")

# Quick sample preview
print("\nSample rows (first 8):")
print(final.head(8).to_string(index=False))


C:\Users\patel\AppData\Local\Temp\ipykernel_2988\3633792695.py:75: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  g['ce_ret_1m_lag1'] = g['atm_ce_ltp'].pct_change(periods=1)
C:\Users\patel\AppData\Local\Temp\ipykernel_2988\3633792695.py:76: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  g['pe_ret_1m_lag1'] = g['atm_pe_ltp'].pct_change(periods=1)
C:\Users\patel\AppData\Local\Temp\ipykernel_2988\3633792695.py:82: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change

Saved: D:\StockMarket\StockMarket\notebooks\F_option_ltp_features_1m.csv
Rows: 7576
CE up 1m: 3679/7576 = 0.486
PE up 1m: 3699/7576 = 0.488

Sample rows (first 8):
                timestamp  expiry  atm_strike                  ce_symbol                  pe_symbol  nifty_ltp  atm_ce_ltp  atm_pe_ltp  nifty_ret_1m  ce_ret_1m  pe_ret_1m  nifty_ret_3m  ce_ret_3m  pe_ret_3m  nifty_ret_5m  ce_ret_5m  pe_ret_5m  ce_up_1m  pe_up_1m  nifty_ret_1m_lag1  ce_ret_1m_lag1  pe_ret_1m_lag1  nifty_ret_3m_rolling_std  nifty_ma_5m  ce_rolling_std_5m  pe_rolling_std_5m  ce_pe_mid  ce_minus_pe  minute_of_day  is_opening_hour  is_closing_hour
2025-11-07 14:13:00+05:30 11Nov25       25450 NSE-NIFTY-11Nov25-25450-CE NSE-NIFTY-11Nov25-25450-PE   25473.70      109.80       72.70      0.000012   0.017304  -0.015818      0.000236  -0.218124   0.286107      0.000120  -0.201730   0.248281         1         0                NaN             NaN             NaN                       NaN   25473.7000                NaN 

In [1]:
import pandas as pd

# ---------------------- INPUT FILES ----------------------
option_file = "Final_option_ltp_1m_rolling.csv"
spot_file = "nifty_spot_1m.csv"

# ---------------------- LOAD FILES -----------------------
df_opt = pd.read_csv(option_file)
df_spot = pd.read_csv(spot_file)

# Convert timestamp to datetime
df_opt['timestamp'] = pd.to_datetime(df_opt['timestamp'])
df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'])

# Sort both files by timestamp
df_opt = df_opt.sort_values('timestamp')
df_spot = df_spot.sort_values('timestamp')

# ---------------------- MERGE ON TIMESTAMP ----------------------
# We merge using outer join to keep all timestamps from both datasets
# then forward fill option values for candles without option tick
df_merged = pd.merge_asof(
    df_spot,
    df_opt,
    on='timestamp',
    direction='nearest',      # take nearest option tick to candle timestamp
    tolerance=pd.Timedelta('1min')   # option data must be within 1 min range
)

# If no matching option tick happens within 1 minute → forward fill
df_merged = df_merged.ffill()

# ---------------------- SAVE RESULT ----------------------
df_merged.to_csv("merged_data.csv", index=False)

print("Merged file saved as merged_data.csv")
print(df_merged.head())


Merged file saved as merged_data.csv
            timestamp      open      high       low     close  volume  \
0 2025-11-07 14:13:00  25472.70  25475.05  25471.65  25473.70     NaN   
1 2025-11-07 14:14:00  25474.35  25476.35  25472.60  25474.00     NaN   
2 2025-11-07 14:15:00  25474.95  25480.40  25469.30  25470.55     NaN   
3 2025-11-07 14:16:00  25470.45  25481.80  25470.45  25479.70     NaN   
4 2025-11-07 14:17:00  25480.20  25483.45  25473.65  25474.60     NaN   

   open_interest  nifty_ltp   expiry  atm_strike                   ce_symbol  \
0            NaN   25473.70  11Nov25       25450  NSE-NIFTY-11Nov25-25450-CE   
1            NaN   25474.00  11Nov25       25450  NSE-NIFTY-11Nov25-25450-CE   
2            NaN   25470.55  11Nov25       25450  NSE-NIFTY-11Nov25-25450-CE   
3            NaN   25479.70  11Nov25       25500  NSE-NIFTY-11Nov25-25500-CE   
4            NaN   25474.60  11Nov25       25450  NSE-NIFTY-11Nov25-25450-CE   

                    pe_symbol  atm_ce_ltp  

In [2]:
# build_merged_features.py
# Usage: python build_merged_features.py
# Inputs (expected in same folder):
#   - option_ltp.csv  (cols: timestamp, nifty_ltp, expiry, atm_strike, ce_symbol, pe_symbol, atm_ce_ltp, atm_pe_ltp)
#   - nifty_spot.csv  (cols: timestamp, open, high, low, close)
# Output:
#   - merged_features_final.csv

import pandas as pd
import numpy as np
from datetime import timedelta

# --------------- CONFIG ---------------
OPTION_FILE = "Final_option_ltp_1m_rolling.csv"
SPOT_FILE   = "nifty_spot_1m.csv"
OUTPUT_FILE = "merged_features_final.csv"

# feature params
SHORT_MOM = 3
MID_MOM = 5
LONG_MOM = 10
RSI_PERIOD = 14
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
VOL_SHORT = 5
VOL_LONG = 20
BREAKOUT_SHORT = 20
BREAKOUT_LONG = 50
HORIZONS = [1, 2, 3]   # minutes ahead
THRESHOLD = 0.001      # 0.1% move

# --------------- HELPERS ---------------
def ema(s, span):
    return s.ewm(span=span, adjust=False).mean()

def rolling_zscore(s, window=50):
    mu = s.rolling(window).mean()
    sd = s.rolling(window).std().replace(0, np.nan)
    return ((s - mu) / sd).fillna(0)

def compute_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0.0)
    down = -1 * delta.clip(upper=0.0)
    roll_up = up.ewm(alpha=1/period, adjust=False).mean()
    roll_down = down.ewm(alpha=1/period, adjust=False).mean()
    rs = roll_up / (roll_down + 1e-12)
    rsi = 100 - (100 / (1 + rs))
    return rsi

def compute_macd(series, fast=12, slow=26, signal=9):
    fast_ema = ema(series, fast)
    slow_ema = ema(series, slow)
    macd = fast_ema - slow_ema
    macd_signal = ema(macd, signal)
    macd_hist = macd - macd_signal
    return macd, macd_signal, macd_hist

# --------------- LOAD & PREP ---------------
# Read CSVs
df_opt = pd.read_csv(OPTION_FILE)
df_spot = pd.read_csv(SPOT_FILE)

# Parse timestamps
df_opt['timestamp'] = pd.to_datetime(df_opt['timestamp'])
df_spot['timestamp'] = pd.to_datetime(df_spot['timestamp'])

# Standardize column names if needed
for c in ['open','high','low','close']:
    if c not in df_spot.columns:
        raise ValueError(f"Spot CSV must contain column '{c}'")

required_opt_cols = ['nifty_ltp','atm_ce_ltp','atm_pe_ltp','ce_symbol','pe_symbol','atm_strike','expiry']
for c in required_opt_cols:
    if c not in df_opt.columns:
        raise ValueError(f"Option CSV must contain column '{c}'")

# Set index and sort
df_spot = df_spot.sort_values('timestamp').set_index('timestamp')
df_opt  = df_opt.sort_values('timestamp').set_index('timestamp')

# Ensure minute frequency continuous timeline (resample to 1min for spot)
# We'll create a continuous index from earliest to latest minute using spot timestamps
start = min(df_spot.index.min(), df_opt.index.min())
end   = max(df_spot.index.max(), df_opt.index.max())
# Align to minute boundaries
start = start.floor('T')
end = end.ceil('T')

full_index = pd.date_range(start=start, end=end, freq='1T')  # 1-minute freq

# Reindex spot to this continuous timeline using minute-level OHLC forward/backfill as appropriate
# If spot already 1-minute candles, reindex will keep them; if missing minutes, we'll forward-fill last candle (option: can also mark as NaN)
df_spot = df_spot.reindex(full_index)

# For missing spot candles: forward fill close/open/high/low with last known candle (keeps continuity)
# Better approach: for missing minute, we set open = prev close, high = max(prev close, next exist), low = min(prev close, next exist), close = prev close
df_spot['close'] = df_spot['close'].ffill()
df_spot['open']  = df_spot['open'].fillna(df_spot['close'])
df_spot['high']  = df_spot['high'].fillna(df_spot['close'])
df_spot['low']   = df_spot['low'].fillna(df_spot['close'])

# Reindex / align options to same timeline using nearest tick within 1 minute
# We'll use merge_asof on sorted indexes (merge_asof requires columns, so reset index)
opt_reset = df_opt.reset_index().rename(columns={'timestamp':'ts'})
spot_reset = df_spot.reset_index().rename(columns={'index':'timestamp'})

# merge_asof requires sorted by key
opt_reset = opt_reset.sort_values('ts')
spot_reset = spot_reset.sort_values('timestamp')

merged = pd.merge_asof(
    spot_reset,
    opt_reset,
    left_on='timestamp',
    right_on='ts',
    direction='nearest',
    tolerance=pd.Timedelta('1min')  # keep option that is within +/- 1 minute
)

# If some rows didn't find an option tick within tolerance, forward-fill option columns so every minute has option values
merged = merged.ffill()

# Clean up: drop helper ts column
if 'ts' in merged.columns:
    merged = merged.drop(columns=['ts'])

# Move timestamp to index
merged['timestamp'] = pd.to_datetime(merged['timestamp'])
merged = merged.set_index('timestamp').sort_index()

# Basic price series
price = merged['close'].astype(float)
ce = merged['atm_ce_ltp'].astype(float)
pe = merged['atm_pe_ltp'].astype(float)

# --------------- FEATURE ENGINEERING ---------------
# Basic returns
merged['nifty_ret_1'] = price.pct_change().fillna(0)
merged['ce_ret_1']    = ce.pct_change().fillna(0)
merged['pe_ret_1']    = pe.pct_change().fillna(0)

# Momentum features
merged[f'mom_{SHORT_MOM}'] = price.pct_change(periods=SHORT_MOM)
merged[f'mom_{MID_MOM}']   = price.pct_change(periods=MID_MOM)
merged[f'mom_{LONG_MOM}']  = price.pct_change(periods=LONG_MOM)

# Volatility (std of returns)
merged['vol_short'] = merged['nifty_ret_1'].rolling(VOL_SHORT).std().fillna(0)
merged['vol_long']  = merged['nifty_ret_1'].rolling(VOL_LONG).std().fillna(0)
merged['vol_spike_ratio'] = (merged['vol_short'] / (merged['vol_long'] + 1e-12)).replace([np.inf, -np.inf], 0).fillna(0)

# ATR using true range (uses spot high/low/close)
high = merged['high'].astype(float).fillna(method='ffill')
low  = merged['low'].astype(float).fillna(method='ffill')
prev_close = merged['close'].shift(1).fillna(method='ffill')
tr1 = (high - low).abs()
tr2 = (high - prev_close).abs()
tr3 = (low - prev_close).abs()
true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
merged['atr_14'] = true_range.rolling(14).mean().fillna(0)

# Breakouts and ranges
merged['rolling_max_20'] = price.rolling(BREAKOUT_SHORT).max()
merged['rolling_min_20'] = price.rolling(BREAKOUT_SHORT).min()
merged['rolling_max_50'] = price.rolling(BREAKOUT_LONG).max()
merged['rolling_min_50'] = price.rolling(BREAKOUT_LONG).min()

merged['breakout_up_20'] = (price > merged['rolling_max_20'].shift(1)).astype(int)
merged['breakout_down_20'] = (price < merged['rolling_min_20'].shift(1)).astype(int)

range_20 = merged['rolling_max_20'] - merged['rolling_min_20']
range_50 = merged['rolling_max_50'] - merged['rolling_min_50']
merged['range_contraction'] = (range_20 / (range_50 + 1e-12)).replace([np.inf, -np.inf], 0).fillna(0)

# RSI
merged['rsi_14'] = compute_rsi(price, period=RSI_PERIOD)

# MACD
macd, macd_signal, macd_hist = compute_macd(price, MACD_FAST, MACD_SLOW, MACD_SIGNAL)
merged['macd'] = macd
merged['macd_signal'] = macd_signal
merged['macd_hist'] = macd_hist

# VWAP proxy: since no volume column in spot, use intraday typical price rolling cumulative per day
typical_price = (merged['high'] + merged['low'] + merged['close']) / 3.0
# compute running daily mean as a VWAP-like proxy
merged['dtt'] = merged.index.date
merged['typ_pv'] = typical_price  # no volume weight available
merged['cum_typ_p'] = merged.groupby('dtt')['typ_pv'].cumsum()
merged['cum_cnt'] = merged.groupby('dtt').cumcount() + 1
merged['vwap'] = (merged['cum_typ_p'] / merged['cum_cnt']).fillna(method='ffill')
merged = merged.drop(columns=['typ_pv','cum_typ_p','cum_cnt','dtt'])

merged['vwap_dist'] = (price - merged['vwap']) / merged['vwap']
merged['vwap_slope'] = merged['vwap'].diff().fillna(0)

# CE/PE features
merged['ce_pe_ratio'] = (ce / (pe + 1e-12)).replace([np.inf, -np.inf], 0).fillna(0)
merged['ce_minus_pe'] = ce - pe
merged['ce_pe_delta'] = (ce.pct_change() - pe.pct_change()).fillna(0)
merged['ce_volatility'] = merged['ce_ret_1'].rolling(VOL_LONG).std().fillna(0)
merged['pe_volatility'] = merged['pe_ret_1'].rolling(VOL_LONG).std().fillna(0)

# Liquidity proxy: use absolute minute returns averaged (since we don't have volume)
merged['nifty_ret_abs'] = merged['nifty_ret_1'].abs().fillna(0)
merged['avg_liq_20'] = merged['nifty_ret_abs'].rolling(20).mean().fillna(0)
merged['liquidity_proxy'] = 1.0 / (merged['avg_liq_20'] + 1e-12)  # higher => more liquid proxy

# Trend filters (ema)
merged['nifty_ema_50'] = ema(price, 50)
merged['nifty_ema_200'] = ema(price, 200)
merged['trend_up'] = (merged['nifty_ema_50'] > merged['nifty_ema_200']).astype(int)
merged['trend_down'] = (merged['nifty_ema_50'] < merged['nifty_ema_200']).astype(int)

# Additional engineered features
merged['momentum_ratio'] = (merged[f'mom_{SHORT_MOM}'] / (merged[f'mom_{LONG_MOM}'].abs() + 1e-12)).replace([np.inf, -np.inf], 0).fillna(0)
merged['volatility_weighted_mom'] = merged['momentum_ratio'] / (merged['vol_long'] + 1e-12)

# Rolling z-scores
merged['z_mom_3'] = rolling_zscore(merged[f'mom_{SHORT_MOM}'], 50)
merged['z_vol_short'] = rolling_zscore(merged['vol_short'], 50)
merged['z_ce_pe_ratio'] = rolling_zscore(merged['ce_pe_ratio'], 50)

# --------------- TARGET LABELS ---------------
for h in HORIZONS:
    up_col = f'target_up_{h}m'
    down_col = f'target_down_{h}m'
    future_price = price.shift(-h)
    ret = (future_price - price) / price
    merged[up_col] = (ret >= THRESHOLD).astype(int)
    merged[down_col] = (ret <= -THRESHOLD).astype(int)

merged['target_1m_direction'] = 0
merged.loc[merged['target_up_1m'] == 1, 'target_1m_direction'] = 1
merged.loc[merged['target_down_1m'] == 1, 'target_1m_direction'] = -1

# --------------- FINAL CLEAN & SAVE ---------------
# Select feature columns (keep important ones)
feature_cols = [
    'open','high','low','close','nifty_ltp','atm_strike','ce_symbol','pe_symbol','atm_ce_ltp','atm_pe_ltp',
    'nifty_ret_1','ce_ret_1','pe_ret_1',
    f'mom_{SHORT_MOM}', f'mom_{MID_MOM}', f'mom_{LONG_MOM}',
    'vol_short','vol_long','vol_spike_ratio','atr_14',
    'breakout_up_20','breakout_down_20','range_contraction',
    'rsi_14','macd','macd_signal','macd_hist',
    'vwap','vwap_dist','vwap_slope',
    'ce_pe_ratio','ce_minus_pe','ce_pe_delta','ce_volatility','pe_volatility',
    'avg_liq_20','liquidity_proxy',
    'nifty_ema_50','nifty_ema_200','trend_up','trend_down',
    'momentum_ratio','volatility_weighted_mom','z_mom_3','z_vol_short','z_ce_pe_ratio'
]

label_cols = [f'target_up_{h}m' for h in HORIZONS] + [f'target_down_{h}m' for h in HORIZONS] + ['target_1m_direction']

# keep only columns present
final_cols = [c for c in feature_cols if c in merged.columns] + [c for c in label_cols if c in merged.columns]

df_final = merged[final_cols].dropna().copy()

# Save final features CSV
df_final.to_csv(OUTPUT_FILE, index=True)
print(f"Saved merged features to {OUTPUT_FILE}")
print("Rows:", len(df_final))
print("Columns:", df_final.columns.tolist())


C:\Users\patel\AppData\Local\Temp\ipykernel_9728\3164520046.py:88: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  start = start.floor('T')
C:\Users\patel\AppData\Local\Temp\ipykernel_9728\3164520046.py:89: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  end = end.ceil('T')
C:\Users\patel\AppData\Local\Temp\ipykernel_9728\3164520046.py:91: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_index = pd.date_range(start=start, end=end, freq='1T')  # 1-minute freq
C:\Users\patel\AppData\Local\Temp\ipykernel_9728\3164520046.py:155: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  high = merged['high'].astype(float).fillna(method='ffill')
C:\Users\patel\AppData\Local\Temp\ipykernel_9728\3164520046.py:156: FutureWarning: Series.fillna with 'method

Saved merged features to merged_features_final.csv
Rows: 40417
Columns: ['open', 'high', 'low', 'close', 'nifty_ltp', 'atm_strike', 'ce_symbol', 'pe_symbol', 'atm_ce_ltp', 'atm_pe_ltp', 'nifty_ret_1', 'ce_ret_1', 'pe_ret_1', 'mom_3', 'mom_5', 'mom_10', 'vol_short', 'vol_long', 'vol_spike_ratio', 'atr_14', 'breakout_up_20', 'breakout_down_20', 'range_contraction', 'rsi_14', 'macd', 'macd_signal', 'macd_hist', 'vwap', 'vwap_dist', 'vwap_slope', 'ce_pe_ratio', 'ce_minus_pe', 'ce_pe_delta', 'ce_volatility', 'pe_volatility', 'avg_liq_20', 'liquidity_proxy', 'nifty_ema_50', 'nifty_ema_200', 'trend_up', 'trend_down', 'momentum_ratio', 'volatility_weighted_mom', 'z_mom_3', 'z_vol_short', 'z_ce_pe_ratio', 'target_up_1m', 'target_up_2m', 'target_up_3m', 'target_down_1m', 'target_down_2m', 'target_down_3m', 'target_1m_direction']


In [6]:
import pandas as pd

# --- CONFIGURATION ---
OPTION_FILE = "Final_option_ltp_1m_rolling.csv"
SPOT_FILE   = "nifty_spot_1m.csv"
OUTPUT_FILE = "nifty_final_clean_1m.csv"

def clean_and_merge():
    print("1. Loading Files...")
    try:
        df_opt = pd.read_csv(OPTION_FILE)
        df_spot = pd.read_csv(SPOT_FILE)
    except FileNotFoundError as e:
        print(f"Error: {e}")
        return

    # 2. Standardize Timestamps (No column name changes)
    print("2. Standardizing Timestamps...")
    def to_dt(series):
        if pd.api.types.is_numeric_dtype(series):
            return pd.to_datetime(series, unit='s')
        return pd.to_datetime(series)

    df_opt['timestamp'] = to_dt(df_opt['timestamp'])
    df_spot['timestamp'] = to_dt(df_spot['timestamp'])

    # 3. Merge Files
    # We bring 'open', 'high', 'low', 'close' from the Spot file.
    # We join on 'timestamp'.
    print("3. Merging Spot OHLC into Option Data...")
    
    # Select only the columns we need from Spot file
    spot_data = df_spot[['timestamp', 'open', 'high', 'low', 'close']]
    
    # Merge (Inner join keeps only matching timestamps)
    df = pd.merge(df_opt, spot_data, on='timestamp', how='inner')

    # 4. Filter Time (Strictly 09:15 to 15:30)
    print("4. Removing extra timestamps (Keeping 09:15 - 15:30)...")
    times = df['timestamp'].dt.time
    start_time = pd.to_datetime("09:15:00").time()
    end_time   = pd.to_datetime("15:30:00").time()
    
    # Keep rows strictly within market hours
    df = df[(times >= start_time) & (times <= end_time)]

    # 5. Save
    print(f"5. Saving to {OUTPUT_FILE}...")
    df.to_csv(OUTPUT_FILE, index=False)
    
    print("-" * 30)
    print(f"✅ SUCCESS! File saved: {OUTPUT_FILE}")
    print(f"Total Rows: {len(df)}")
    print("Columns:", list(df.columns))
    print(df.head())

if __name__ == "__main__":
    clean_and_merge()

1. Loading Files...
2. Standardizing Timestamps...
3. Merging Spot OHLC into Option Data...
4. Removing extra timestamps (Keeping 09:15 - 15:30)...
5. Saving to nifty_final_clean_1m.csv...
------------------------------
✅ SUCCESS! File saved: nifty_final_clean_1m.csv
Total Rows: 7598
Columns: ['timestamp', 'nifty_ltp', 'expiry', 'atm_strike', 'ce_symbol', 'pe_symbol', 'atm_ce_ltp', 'atm_pe_ltp', 'open', 'high', 'low', 'close']
            timestamp  nifty_ltp   expiry  atm_strike  \
0 2025-11-07 14:13:00   25473.70  11Nov25       25450   
1 2025-11-07 14:14:00   25474.00  11Nov25       25450   
2 2025-11-07 14:15:00   25470.55  11Nov25       25450   
3 2025-11-07 14:16:00   25479.70  11Nov25       25500   
4 2025-11-07 14:17:00   25474.60  11Nov25       25450   

                    ce_symbol                   pe_symbol  atm_ce_ltp  \
0  NSE-NIFTY-11Nov25-25450-CE  NSE-NIFTY-11Nov25-25450-PE      109.80   
1  NSE-NIFTY-11Nov25-25450-CE  NSE-NIFTY-11Nov25-25450-PE      111.70   
2  NSE-

In [7]:
import pandas as pd
import numpy as np

# ---------------- CONFIG ----------------
INPUT = "nifty_final_clean_1m.csv"
OUTPUT = "features_final.csv"

RSI_PERIOD = 14
MACD_FAST = 12
MACD_SLOW = 26
MACD_SIGNAL = 9
SHORT_MOM = 3
MID_MOM = 5
LONG_MOM = 10
VOL_SHORT = 5
VOL_LONG = 20
BREAKOUT_SHORT = 20
BREAKOUT_LONG = 50
HORIZONS = [1, 2, 3]
THRESHOLD = 0.001  # 0.1%

# ---------------- HELPERS ----------------
def ema(s, span):
    return s.ewm(span=span, adjust=False).mean()

def compute_rsi(series, period=14):
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -1 * delta.clip(upper=0)
    roll_up = up.ewm(alpha=1/period, adjust=False).mean()
    roll_down = down.ewm(alpha=1/period, adjust=False).mean()
    rs = roll_up / (roll_down + 1e-12)
    return 100 - (100 / (1 + rs))

def compute_macd(series, fast, slow, signal):
    fast_ema = ema(series, fast)
    slow_ema = ema(series, slow)
    macd = fast_ema - slow_ema
    macd_signal = ema(macd, signal)
    macd_hist = macd - macd_signal
    return macd, macd_signal, macd_hist

# ---------------- LOAD ----------------
df = pd.read_csv(INPUT)
df["timestamp"] = pd.to_datetime(df["timestamp"])
df = df.set_index("timestamp")
df = df.sort_index()

# PRICE SERIES
price = df["close"]
ce = df["atm_ce_ltp"]
pe = df["atm_pe_ltp"]

# ---------------- BASIC RETURNS ----------------
df["nifty_ret_1"] = price.pct_change().fillna(0)
df["ce_ret_1"] = ce.pct_change().fillna(0)
df["pe_ret_1"] = pe.pct_change().fillna(0)

# ---------------- MOMENTUM ----------------
df[f"mom_{SHORT_MOM}"] = price.pct_change(SHORT_MOM)
df[f"mom_{MID_MOM}"] = price.pct_change(MID_MOM)
df[f"mom_{LONG_MOM}"] = price.pct_change(LONG_MOM)

# ---------------- VOLATILITY ----------------
df["vol_short"] = df["nifty_ret_1"].rolling(VOL_SHORT).std().fillna(0)
df["vol_long"] = df["nifty_ret_1"].rolling(VOL_LONG).std().fillna(0)
df["vol_spike_ratio"] = (df["vol_short"] / (df["vol_long"] + 1e-12)).replace([np.inf, -np.inf], 0)

# ---------------- ATR ----------------
high = df["high"]
low = df["low"]
prev_close = price.shift(1)

tr = pd.concat([
    (high - low).abs(),
    (high - prev_close).abs(),
    (low - prev_close).abs()
], axis=1).max(axis=1)

df["atr_14"] = tr.rolling(14).mean().fillna(0)

# ---------------- BREAKOUTS ----------------
df["rolling_max_20"] = price.rolling(BREAKOUT_SHORT).max()
df["rolling_min_20"] = price.rolling(BREAKOUT_SHORT).min()

df["breakout_up_20"] = (price > df["rolling_max_20"].shift(1)).astype(int)
df["breakout_down_20"] = (price < df["rolling_min_20"].shift(1)).astype(int)

df["range_contraction"] = (
    (df["rolling_max_20"] - df["rolling_min_20"]) /
    ((df["rolling_max_20"].rolling(BREAKOUT_LONG).max() -
      df["rolling_min_20"].rolling(BREAKOUT_LONG).min()) + 1e-12)
).fillna(0)

# ---------------- RSI ----------------
df["rsi_14"] = compute_rsi(price, RSI_PERIOD)

# ---------------- MACD ----------------
df["macd"], df["macd_signal"], df["macd_hist"] = compute_macd(price, MACD_FAST, MACD_SLOW, MACD_SIGNAL)

# ---------------- VWAP PROXY ----------------
tp = (df["high"] + df["low"] + df["close"]) / 3
df["vwap"] = tp.rolling(20).mean()  # proxy
df["vwap_dist"] = (price - df["vwap"]) / df["vwap"]
df["vwap_slope"] = df["vwap"].diff().fillna(0)

# ---------------- CE/PE FEATURES ----------------
df["ce_pe_ratio"] = ce / (pe + 1e-12)
df["ce_minus_pe"] = ce - pe
df["ce_pe_delta"] = ce.pct_change() - pe.pct_change()
df["ce_volatility"] = df["ce_ret_1"].rolling(VOL_LONG).std().fillna(0)
df["pe_volatility"] = df["pe_ret_1"].rolling(VOL_LONG).std().fillna(0)

# ---------------- LIQUIDITY PROXY ----------------
df["avg_liq_20"] = df["nifty_ret_1"].abs().rolling(20).mean().fillna(0)
df["liquidity_proxy"] = 1 / (df["avg_liq_20"] + 1e-12)

# ---------------- TREND FILTER ----------------
df["nifty_ema_50"] = ema(price, 50)
df["nifty_ema_200"] = ema(price, 200)
df["trend_up"] = (df["nifty_ema_50"] > df["nifty_ema_200"]).astype(int)
df["trend_down"] = (df["nifty_ema_50"] < df["nifty_ema_200"]).astype(int)

# ---------------- ADVANCED FEATURES ----------------
df["momentum_ratio"] = df[f"mom_{SHORT_MOM}"] / (df[f"mom_{LONG_MOM}"].abs() + 1e-12)
df["volatility_weighted_mom"] = df["momentum_ratio"] / (df["vol_long"] + 1e-12)

df["z_mom_3"] = (df[f"mom_{SHORT_MOM}"] - df[f"mom_{SHORT_MOM}"].rolling(50).mean()) / \
                (df[f"mom_{SHORT_MOM}"].rolling(50).std() + 1e-12)

df["z_vol_short"] = (df["vol_short"] - df["vol_short"].rolling(50).mean()) / \
                    (df["vol_short"].rolling(50).std() + 1e-12)

df["z_ce_pe_ratio"] = (df["ce_pe_ratio"] - df["ce_pe_ratio"].rolling(50).mean()) / \
                      (df["ce_pe_ratio"].rolling(50).std() + 1e-12)

# ---------------- TARGETS ----------------
for h in HORIZONS:
    future_price = price.shift(-h)
    ret = (future_price - price) / price
    df[f"target_up_{h}m"] = (ret >= THRESHOLD).astype(int)
    df[f"target_down_{h}m"] = (ret <= -THRESHOLD).astype(int)

df["target_1m_direction"] = 0
df.loc[df["target_up_1m"] == 1, "target_1m_direction"] = 1
df.loc[df["target_down_1m"] == 1, "target_1m_direction"] = -1

# ---------------- CLEAN ----------------
df = df.dropna()

# ---------------- SAVE ----------------
df.to_csv(OUTPUT)
print("Feature extraction completed →", OUTPUT)
print(df.head())


C:\Users\patel\AppData\Local\Temp\ipykernel_9728\2142607532.py:56: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["ce_ret_1"] = ce.pct_change().fillna(0)
C:\Users\patel\AppData\Local\Temp\ipykernel_9728\2142607532.py:110: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df["ce_pe_delta"] = ce.pct_change() - pe.pct_change()


Feature extraction completed → features_final.csv
                     nifty_ltp   expiry  atm_strike  \
timestamp                                             
2025-11-07 15:05:00   25479.95  11Nov25       25500   
2025-11-07 15:06:00   25475.90  11Nov25       25500   
2025-11-07 15:07:00   25479.85  11Nov25       25500   
2025-11-07 15:08:00   25482.75  11Nov25       25500   
2025-11-07 15:09:00   25482.15  11Nov25       25500   

                                      ce_symbol                   pe_symbol  \
timestamp                                                                     
2025-11-07 15:05:00  NSE-NIFTY-11Nov25-25500-CE  NSE-NIFTY-11Nov25-25500-PE   
2025-11-07 15:06:00  NSE-NIFTY-11Nov25-25500-CE  NSE-NIFTY-11Nov25-25500-PE   
2025-11-07 15:07:00  NSE-NIFTY-11Nov25-25500-CE  NSE-NIFTY-11Nov25-25500-PE   
2025-11-07 15:08:00  NSE-NIFTY-11Nov25-25500-CE  NSE-NIFTY-11Nov25-25500-PE   
2025-11-07 15:09:00  NSE-NIFTY-11Nov25-25500-CE  NSE-NIFTY-11Nov25-25500-PE   

          

In [8]:
# update_targets_0p05.py
# Creates target_up_1m_0p05 and target_1m_direction_0p05 in merged_features_final.csv
import pandas as pd
import numpy as np

INPUT = "features_final.csv"
OUTPUT = "features_final_with_target.csv"

THRESHOLD = 0.0005  # 0.05% = ~10 points at NIFTY 20k-22k
HORIZON = 1  # minutes

df = pd.read_csv(INPUT)
if 'timestamp' in df.columns:
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp').sort_index()

# use 'close' as base price
price = df['close'].astype(float)

# target up if future price increases at least THRESHOLD in HORIZON minutes
future_price = price.shift(-HORIZON)
ret = (future_price - price) / price

df[f"target_up_{HORIZON}m_0p05"] = (ret >= THRESHOLD).astype(int)
df[f"target_down_{HORIZON}m_0p05"] = (ret <= -THRESHOLD).astype(int)

# direction label
df[f"target_1m_direction_0p05"] = 0
df.loc[df[f"target_up_{HORIZON}m_0p05"] == 1, f"target_1m_direction_0p05"] = 1
df.loc[df[f"target_down_{HORIZON}m_0p05"] == 1, f"target_1m_direction_0p05"] = -1

# drop final rows with NaN targets (warmup at end)
df = df.dropna()

# summary
total = len(df)
pos = df[f"target_up_{HORIZON}m_0p05"].sum()
neg = df[f"target_down_{HORIZON}m_0p05"].sum()
print("Rows:", total)
print("Target up positives:", pos, "Rate:", pos/total)
print("Target down negatives:", neg, "Rate:", neg/total)

# save
df.to_csv(OUTPUT, index=True)
print("Saved with new targets to", OUTPUT)


Rows: 7496
Target up positives: 131 Rate: 0.01747598719316969
Target down negatives: 130 Rate: 0.017342582710779084
Saved with new targets to features_final_with_target.csv


In [2]:
import pandas as pd

df = pd.read_csv("features_final.csv", parse_dates=["timestamp"])
df = df.sort_values("timestamp")

# 1-minute absolute returns
df["ret_1m_abs"] = (df["close"].shift(-1) - df["close"]).abs()

# Average movement
avg_points = df["ret_1m_abs"].mean()
median_points = df["ret_1m_abs"].median()

# Percent returns
df["ret_1m_pct"] = df["ret_1m_abs"] / df["close"]

avg_pct = df["ret_1m_pct"].mean()
median_pct = df["ret_1m_pct"].median()

print("Average 1m movement (points):", avg_points)
print("Median 1m movement (points):", median_points)
print("Average 1m movement (%):", avg_pct)
print("Median 1m movement (%):", median_pct)


Average 1m movement (points): 4.037991994663116
Median 1m movement (points): 2.849999999998545
Average 1m movement (%): 0.00015537391192760994
Median 1m movement (%): 0.00010902705783784214


In [3]:
print(df["ret_1m_abs"].describe())


count    7495.000000
mean        4.037992
std         4.965033
min         0.000000
25%         1.300000
50%         2.850000
75%         5.300000
max       144.300000
Name: ret_1m_abs, dtype: float64


In [2]:
import time
import pandas as pd
from datetime import timedelta
from growwapi import GrowwAPI

# --- CREDENTIALS ---
API_KEY = "eyJraWQiOiJaTUtjVXciLCJhbGciOiJFUzI1NiJ9.eyJleHAiOjI1NTM1Njg0NzYsImlhdCI6MTc2NTE2ODQ3NiwibmJmIjoxNzY1MTY4NDc2LCJzdWIiOiJ7XCJ0b2tlblJlZklkXCI6XCJhMTg3NDVhMy1hN2M1LTRlOTQtODE1MS1lZjUxZDQ5OGE2Y2RcIixcInZlbmRvckludGVncmF0aW9uS2V5XCI6XCJlMzFmZjIzYjA4NmI0MDZjODg3NGIyZjZkODQ5NTMxM1wiLFwidXNlckFjY291bnRJZFwiOlwiMDdmMDA0MGMtZTk4Zi00ZDNmLTk5Y2EtZDc1ZjBlYWU5M2NlXCIsXCJkZXZpY2VJZFwiOlwiZDMyMWIxMzUtZWQ5Mi01ZWJkLWJjMDUtZTY1NDY2OWRiMDM5XCIsXCJzZXNzaW9uSWRcIjpcIjBlOWMyYWZmLTM0NzktNDUyMi1iODE4LTczNTZlMzFkYmY1Y1wiLFwiYWRkaXRpb25hbERhdGFcIjpcIno1NC9NZzltdjE2WXdmb0gvS0EwYk1yOE5XVzhzdTNvZ080am1ZUzIwZEpSTkczdTlLa2pWZDNoWjU1ZStNZERhWXBOVi9UOUxIRmtQejFFQisybTdRPT1cIixcInJvbGVcIjpcImF1dGgtdG90cFwiLFwic291cmNlSXBBZGRyZXNzXCI6XCIyNDA5OjQwOTA6MTA4ZjpkYzA1OjNkMWQ6MWZmMDo1YWFjOjYwNTYsMTcyLjcxLjE5OC4xOSwzNS4yNDEuMjMuMTIzXCIsXCJ0d29GYUV4cGlyeVRzXCI6MjU1MzU2ODQ3NjQzNn0iLCJpc3MiOiJhcGV4LWF1dGgtcHJvZC1hcHAifQ.VuAMgqoC3e32gduObByNz97jFfG-ikXoREum26XPkvyMpj9JgCedXBI81jxGTPTrZD9i1wIL0s38LPd9vc9ApA"
API_SECRET = "xy0sbQ4r*!HN3&&UKc9vpwti4xx8PR)("

def auth():
    try:
        token = GrowwAPI.get_access_token(api_key=API_KEY, secret=API_SECRET)
        return GrowwAPI(token)
    except Exception as e:
        print(f"Auth failed: {e}")
        exit()

groww = auth()
print("Logged into Groww!")
SYMBOL = "NSE-NIFTY" 
EXCHANGE = "NSE"          # Passed as string
SEGMENT = "CASH"       # Passed as string ("INDICES" for Nifty 50, "CASH" for stocks)
           # Passed as string (e.g., "1m", "5m", "15m")

# --- CHUNKING LOGIC ---
total_days = 3994
chunk_size_days = 5
end_date = pd.Timestamp.now()
start_date = end_date - timedelta(days=total_days)

all_candles = []
current_start = start_date

INTERVALS = ["1hour"]

for INTERVAL in INTERVALS:
    all_candles = []
    current_start = start_date
    print("Downloading interval:", INTERVAL)
    while current_start < end_date:
        current_end = current_start + timedelta(days=chunk_size_days)
        if current_end > end_date:
            current_end = end_date
        s_str = current_start.strftime("%Y-%m-%d %H:%M:%S")
        e_str = current_end.strftime("%Y-%m-%d %H:%M:%S")
        try:
            resp = groww.get_historical_candles(
                exchange=EXCHANGE,
                segment=SEGMENT,
                groww_symbol=SYMBOL,
                start_time=s_str,
                end_time=e_str,
                candle_interval=INTERVAL
            )
            if resp and "candles" in resp and resp["candles"]:
                all_candles.extend(resp["candles"])
        except Exception as e:
            print("Error:", e)
        current_start = current_end
        time.sleep(0.5)

    if all_candles:
        df = pd.DataFrame(all_candles)
        # same robust timestamp fix as you already have...
        # save file name by interval
        df.to_csv(f"nifty_spot_{INTERVAL}.csv", index=False)
        print("Saved", f"nifty_spot_{INTERVAL}.csv")


Ready to Groww!
Logged into Groww!
Saved nifty_spot_1hour.csv
